In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv',low_memory=True)
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv',low_memory=True)
sample = pd.read_csv('/kaggle/input/playground-series-s5e4/sample_submission.csv',low_memory=True)

In [3]:
train.head(8)

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
5,5,Fitness First,Episode 19,26.54,Health,48.96,Saturday,Afternoon,NaN,3.0,Positive,22.77047
6,6,Criminal Minds,Episode 47,69.83,True Crime,35.82,Sunday,Night,39.02,0.0,Neutral,64.75024
7,7,News Roundup,Episode 44,48.52,News,44.99,Thursday,Night,20.12,0.0,Positive,22.37517


In [5]:
cat_cols = train.select_dtypes(include='object').columns

In [6]:
cat_cols

Index(['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day',
       'Publication_Time', 'Episode_Sentiment'],
      dtype='object')

In [9]:
def data_process(df):
    df['Episode_Title_num'] = df['Episode_Title'].astype(str).str.replace('Episode ', '').astype(int)
    df['Guest_Popularity_percentage'].fillna(df['Guest_Popularity_percentage'].median(), inplace=True)
    df['Episode_Length_minutes'].fillna(df['Episode_Length_minutes'].median(), inplace=True)
    df['Number_of_Ads'].fillna(df['Number_of_Ads'].median(), inplace=True)
    df['Guest_Popularity_percentage'] = df['Guest_Popularity_percentage'].clip(upper=3)
    df['Episode_Sentiment'] = df['Episode_Sentiment'].replace({'Neutral': 0, 'Positive': 1, 'Negative': -1})

    df['Ad_Density'] = df['Number_of_Ads'] / (df['Episode_Length_minutes'] + 1e-3)
    df['Popularity_Diff'] = df['Host_Popularity_percentage'] - df['Guest_Popularity_percentage']
    df['Popularity_Interaction'] = df['Host_Popularity_percentage'] * df['Guest_Popularity_percentage']
    df['Host_Popularity_squared'] = df['Host_Popularity_percentage'] ** 2
    df['Popularity_Average'] = (df['Host_Popularity_percentage'] + df['Guest_Popularity_percentage'])/2
    
    return df


In [11]:
train_data = data_process(train)
test_data = data_process(test)

<ipython-input-9-aa4fd69f70bc>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Guest_Popularity_percentage'].fillna(df['Guest_Popularity_percentage'].median(), inplace=True)
<ipython-input-9-aa4fd69f70bc>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing

In [12]:
train_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,0,Mystery Matters,Episode 98,63.84,True Crime,74.81,Thursday,Night,3.0,0.0,1,31.41998,98,0.000000,71.81,224.43,5596.5361,38.905
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,3.0,2.0,-1,88.01241,26,0.016694,63.95,200.85,4482.3025,34.975
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,3.0,0.0,-1,44.92531,16,0.000000,66.97,209.91,4895.8009,36.485
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,3.0,2.0,1,46.27824,45,0.029775,54.22,171.66,3274.1284,30.110
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,3.0,3.0,0,75.61031,86,0.027147,77.07,240.21,6411.2049,41.535


In [13]:
test_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,3.0,1.0,0,73,0.012664,35.11,114.33,1452.3721,20.555
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,3.0,0.0,0,23,0.000000,68.29,213.87,5082.2641,37.145
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,3.0,0.0,1,11,0.000000,64.89,203.67,4609.0521,35.445
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,3.0,2.0,1,73,0.017332,20.40,70.20,547.5600,13.200
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,3.0,2.0,0,50,0.027654,55.10,174.30,3375.6100,30.550


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
cat_cols = train_data.select_dtypes(exclude=['number']).columns.tolist()
train_data[cat_cols] = train_data[cat_cols].fillna(train_data[cat_cols].mode().iloc[0])
test_data[cat_cols] = test_data[cat_cols].fillna(test_data[cat_cols].mode().iloc[0])

print(cat_cols)
for col in cat_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time']


In [17]:
train_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,0,34,98,63.84,9,74.81,4,3,3.0,0.0,1,31.41998,98,0.000000,71.81,224.43,5596.5361,38.905
1,1,24,19,119.80,1,66.95,2,0,3.0,2.0,-1,88.01241,26,0.016694,63.95,200.85,4482.3025,34.975
2,2,40,8,73.90,2,69.97,5,1,3.0,0.0,-1,44.92531,16,0.000000,66.97,209.91,4895.8009,36.485
3,3,10,40,67.17,8,57.22,1,2,3.0,2.0,1,46.27824,45,0.029775,54.22,171.66,3274.1284,30.110
4,4,31,85,110.51,3,80.07,1,0,3.0,3.0,0,75.61031,86,0.027147,77.07,240.21,6411.2049,41.535


In [18]:
test_data.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,750000,11,71,78.96,2,38.11,2,1,3.0,1.0,0,73,0.012664,35.11,114.33,1452.3721,20.555
1,750001,36,16,27.87,5,71.29,3,2,3.0,0.0,0,23,0.000000,68.29,213.87,5082.2641,37.145
2,750002,24,3,69.10,1,67.89,0,1,3.0,0.0,1,11,0.000000,64.89,203.67,4609.0521,35.445
3,750003,4,71,115.39,1,23.40,3,2,3.0,2.0,1,73,0.017332,20.40,70.20,547.5600,13.200
4,750004,27,46,72.32,4,58.10,6,2,3.0,2.0,0,50,0.027654,55.10,174.30,3375.6100,30.550


In [19]:
train_data.to_csv('final_train.csv',index = False)
test_data.to_csv('final_test.csv',index = False)